<a href="https://colab.research.google.com/github/yuichi0625/cellar/blob/main/check_country_language_pair_on_transformers/bert_large_uncased_english.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.0MB 6.3MB/s 
     |████████████████████████████████| 890kB 41.9MB/s 
     |████████████████████████████████| 3.2MB 33.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=ecd70043aad150bcc4123679488c06cc1c2a6ac1cab3f4547aa13bfe2420fd0f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!wget https://raw.githubusercontent.com/yuichi0625/cellar/main/check_country_language_pair_on_transformers/countries.csv

--2021-03-23 16:26:59--  https://raw.githubusercontent.com/yuichi0625/cellar/main/check_country_language_pair_on_transformers/countries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7068 (6.9K) [text/plain]
Saving to: ‘countries.csv’

countries.csv       100%[===================>]   6.90K  --.-KB/s    in 0s      

2021-03-23 16:26:59 (89.3 MB/s) - ‘countries.csv’ saved [7068/7068]



In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
pd.set_option('display.max_rows', 500)

tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')
model = AutoModelForMaskedLM.from_pretrained('bert-large-uncased')

sequence = 'I live in {} and I speak {}.'

rows = []
for name in pd.read_csv('countries.csv')['en'].sort_values():
  input = tokenizer.encode(sequence.format(name, tokenizer.mask_token), return_tensors='pt')
  mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

  token_logits = model(input).logits[0]
  mask_token_logits = token_logits[mask_token_index, :][0]
  mask_token_probs = torch.softmax(mask_token_logits, dim=0)

  candidate_indices = torch.topk(mask_token_logits, k=3, dim=0).indices.tolist()
  result = [f'{tokenizer.decode([ci])} ({mask_token_probs[ci] * 100:.0f}%)' for ci in candidate_indices]

  rows.append([name, *result])

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
pd.DataFrame(rows, columns=['country', 'language(top1)', 'language(top2)', 'language(top3)'])

,country,language(top1),language(top2),language(top3)
0,Afghanistan,persian (56%),arabic (17%),english (14%)
1,Albania,albanian (95%),english (1%),italian (1%)
2,Algeria,french (47%),arabic (46%),english (4%)
3,Andorra,spanish (72%),french (16%),english (5%)
4,Angola,portuguese (89%),english (4%),spanish (3%)
5,Antigua and Barbuda,english (96%),spanish (2%),french (1%)
6,Argentina,spanish (65%),english (27%),portuguese (3%)
7,Armenia,armenian (87%),russian (6%),english (2%)
8,Australia,english (72%),french (4%),japanese (3%)
9,Austria,german (79%),hungarian (8%),english (3%)
